## 상장기업 재무정보 - 다중회사 주요 재무지표
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/

### 1. 대상 회사/연도/보고서 설정 및 주요 재무지표 정보 가져오기

> 보고서코드
* 1분기보고서 : 11013
* 반기보고서 : 11012
* 3분기보고서 : 11014
* 사업보고서 : 11011

> 지표분류코드
* 수익성지표 : M210000
* 안정성지표 : M220000
* 성장성지표 : M230000
* 활동성지표 : M240000

In [1]:
import requests
import pandas as pd

fiscal_year = '2023'
report_code = '11014'  

# 재무지표
idx_classes = ['M210000', 'M220000', 'M230000', 'M240000']

# 비교 대상 회사(순서별)
dict_corp_codes = {
    '00787376' : '대성에너지',
    '00130684' : '서울도시가스',
    '00128555' : '삼천리',
    '01208885' : '경동도시가스'
}

In [2]:
""" 회사별 재무지표 정보 가져오기 (json format)
※ 2023년 3분기 이후 부터 정보제공
"""

url = 'https://opendart.fss.or.kr/api/fnlttCmpnyIndx.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'

dict_fi = {}

for corp_code in dict_corp_codes.keys():
    df_fi_all = pd.DataFrame()
    for idx_class in idx_classes:
        params = {
            'corp_code': corp_code,
            'bsns_year': fiscal_year,
            'reprt_code': report_code,
            'idx_cl_code': idx_class,            # OFS:재무제표
            'crtfc_key': api_key,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:             # URL GET 200 : 정상
            print("URL GET Error")
        if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
            # 데이터프레임 형태로 변환
            df_fi = pd.json_normalize(response.json()['list'])
            df_fi_all = pd.concat([df_fi_all, df_fi], axis=0, ignore_index=True)   # 합치기(행 기준)
            
    if not df_fi_all.empty:
        dict_fi[corp_code] = df_fi_all

In [3]:
len(dict_fi)

4

In [4]:
""" 재무지표 Category """
df_category = pd.DataFrame()
# df_category.columns = ['reprt_code', 'bsns_year', 'idx_cl_code', 'idx_cl_nm', 'idx_code', 'idx_nm']
for k, df in dict_fi.items():  
    df_temp = df[['reprt_code', 'bsns_year', 'idx_cl_code', 'idx_cl_nm', 'idx_code', 'idx_nm']]
    df_category = pd.concat([df_category, df_temp], axis=0, ignore_index=True)   # 합치기(행 기준)
df_category.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [5]:
len(df_category)

66

In [6]:
df_category[['idx_cl_code', 'idx_cl_nm']].value_counts()

idx_cl_code  idx_cl_nm
M220000      안정성지표        22
M230000      성장성지표        18
M210000      수익성지표        15
M240000      활동성지표        11
Name: count, dtype: int64

In [7]:
""" 회사별 비교 """
df_values = df_category
for k, df in dict_fi.items():  
    df_val = df[['idx_code', 'idx_val']]
    corp_name = dict_corp_codes[k]
    df_val.columns = ['idx_code', corp_name]    # idx_val 컬럼명을 corp_code로 변경
    # # 열 병합
    df_values = pd.merge(left=df_values, right=df_val, how='left', on='idx_code')

In [8]:
df_values

,reprt_code,bsns_year,idx_cl_code,idx_cl_nm,idx_code,idx_nm,대성에너지,서울도시가스,삼천리,경동도시가스
0,11014,2023,M210000,수익성지표,M211100,세전계속사업이익률,NaN,NaN,NaN,NaN
1,11014,2023,M210000,수익성지표,M211200,순이익률,0.826,1.666,3.46,1.935
2,11014,2023,M210000,수익성지표,M211250,총포괄이익률,0.826,NaN,3.566,2.134
3,11014,2023,M210000,수익성지표,M211300,매출총이익률,13.357,12.743,10.353,6.981
4,11014,2023,M210000,수익성지표,M211400,매출원가율,86.643,87.257,89.647,93.019
...,...,...,...,...,...,...,...,...,...,...
61,11014,2023,M240000,활동성지표,M241600,유형자산회전율,142.604,207.916,183.152,400.382
62,11014,2023,M240000,활동성지표,M241800,타인자본회전율,162.528,183.482,138.081,267.167
63,11014,2023,M240000,활동성지표,M241900,자기자본회전율,220.279,109.223,257.175,318.045
64,11014,2023,M240000,활동성지표,M242000,자본금회전율,2509.143,3312.916,#########,8765.417


In [9]:
# 각 회사별로 가져온 데이터를 하나의 엑셀 파일에 각각의 시트로 생성
filename = ".".join([fiscal_year + "_" + report_code, "xlsx"])
path = "D:/PythonProject/data-gatherer/dart/data/" + filename
with pd.ExcelWriter(path) as writer:
    df_values.to_excel(writer, sheet_name="ALL")
    # 각 회사별로 데이터 저장
    for k, df in dict_fi.items():
        df.to_excel(writer, sheet_name=k)